In [ ]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 843.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=638746e776c8f0e872d7af19ad22cf5ce5a4aa003bd2960e219af12fa387600e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import json
import datasets
from datasets import Dataset, ClassLabel, Value, Features, Sequence
import os

In [ ]:
def load_data(directory):
    texts, labels = [], []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            for item in data['annotations']:
                if item == None:
                  continue
                text = item[0]
                entities = item[1]['entities']
                text_labels = ['O'] * len(text)  # Default label
                for start, end, label in entities:
                    if label == "QUESTION":
                        text_labels[start] = 'B-QUESTION'
                        for i in range(start + 1, end):
                            text_labels[i] = 'I-QUESTION'
                    # Assuming you do not want to include 'GARBAGE'
                texts.append(text)
                labels.append(text_labels)
    return texts, labels

In [ ]:
import re
def create_dataset(texts, labels):
    # Convert list of characters in texts to list of words (assuming words were intended)
    tokenized_texts = [re.findall(r'\S+|\s', text) for text in texts]  # Example of simple whitespace tokenization
    # Adjust labels to match the tokenization
    adjusted_labels = []
    for text, label in zip(tokenized_texts, labels):
        word_labels = []
        word_index = 0
        for word in text:
            if word_index < len(label):
                word_labels.append(label[word_index])
            word_index += len(word)  # Adjust based on actual tokenization
        adjusted_labels.append(word_labels)

    # Define features with correct types
    features = Features({
        'tokens': Sequence(Value('string')),
        'ner_tags': Sequence(ClassLabel(names=['O', 'B-QUESTION', 'I-QUESTION']))
    })

    # Create a dictionary with the structured data
    data = {'tokens': tokenized_texts, 'ner_tags': adjusted_labels}
    # Print the tokens
    print(data['tokens'])
    return Dataset.from_dict(data, features=features)

In [ ]:
# Load and create datasets
directory = r"/NERjson"

In [ ]:
texts, labels = load_data(directory)

In [ ]:
dataset = create_dataset(texts, labels)

[['Seção', ' ', 'BEM', ' ', '-', ' ', 'Escala', ' ', 'de', ' ', 'bem-estar', ' ', 'mental', ' ', 'Warwick-Edinbrugh', '\r'], ['BEM1', ' ', 'Tenho', ' ', 'me', ' ', 'sentido', ' ', 'otimista', ' ', 'com', ' ', 'relação', ' ', 'ao', ' ', 'futuro', ' ', '(', ' ', ' ', ')', ' ', '1', ' ', ' ', '(', ' ', ' ', ')', ' ', '2', ' ', '(', ' ', ' ', ')', ' ', '3', ' ', '(', ' ', ' ', ')', ' ', '4', ' ', '(', ' ', ' ', ')', ' ', '5', '\r'], ['Marque', ' ', 'com', ' ', 'um', ' ', '(X),', ' ', 'por', ' ', 'favor,', ' ', 'cada', ' ', 'uma', ' ', 'das', ' ', 'afirmações,', ' ', 'a', ' ', 'que', ' ', 'se', ' ', 'aproxima', ' ', 'mais', ' ', 'do', ' ', 'modo', ' ', 'como', ' ', 'você', ' ', 'tem', ' ', 'se', ' ', 'sentido', ' ', 'nas', ' ', 'últimas', ' ', 'duas', ' ', 'semanas.', '\r'], ['Abaixo', ' ', 'se', ' ', 'encontram', ' ', 'algumas', ' ', 'afirmações', ' ', 'sobre', ' ', 'sentimentos', ' ', 'e', ' ', 'pensamentos.', '\r'], ['BEM6', ' ', 'Tenho', ' ', 'me', ' ', 'sentido', ' ', 'próximo(a)', ' '

In [ ]:
example = dataset[15]
tokens = example["tokens"]
ner_tags = example["ner_tags"]

print("Tokens:", tokens)
print("NER Tags:", ner_tags)

Tokens: ['4.', ' ', 'I', ' ', 'was', ' ', 'very', ' ', 'restless.', '\r']
NER Tags: [0, 0, 1, 2, 2, 2, 2, 2, 2, 0]


In [ ]:
dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 2902
})

In [ ]:
dataset.features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-QUESTION', 'I-QUESTION'], id=None), length=-1, id=None)

In [ ]:
dataset.description

''

In [ ]:
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
example_text = dataset[0]

tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'Se',
 '##ção',
 'BE',
 '##M',
 '-',
 'Es',
 '##cala',
 'de',
 'bem',
 '-',
 'estar',
 'mental',
 'Warwick',
 '-',
 'Ed',
 '##in',
 '##brug',
 '##h',
 '[SEP]']

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Example usage
examples = {
    'tokens': [["Hello", "world"], ["Test", "sentence"]],
    'ner_tags': [[0, 1], [0, 1]]  # Assuming simple numeric labels for demonstration
}

tokenized_and_aligned = tokenize_and_align_labels(examples)
tokenized_and_aligned

{'input_ids': [[101, 31178, 11356, 102], [101, 22495, 49219, 102]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[-100, 0, 1, -100], [-100, 0, 1, -100]]}

In [ ]:
q = tokenize_and_align_labels(dataset[7:8])
print(q)

{'input_ids': [[101, 124, 119, 146, 24666, 10380, 71841, 10162, 146, 12820, 20694, 12166, 10111, 12172, 33338, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100]]}


In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
3_______________________________________ 0
._______________________________________ 0
I_______________________________________ 1
felt____________________________________ 2
so______________________________________ 2
tire____________________________________ 2
##d_____________________________________ 2
I_______________________________________ 2
just____________________________________ 2
sat_____________________________________ 2
around__________________________________ 2
and_____________________________________ 2
did_____________________________________ 2
nothing_________________________________ 2
._______________________________________ 2
[SEP]___________________________________ -100


In [ ]:
# Apply function
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2902 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets[0]

{'tokens': ['Seção',
  ' ',
  'BEM',
  ' ',
  '-',
  ' ',
  'Escala',
  ' ',
  'de',
  ' ',
  'bem-estar',
  ' ',
  'mental',
  ' ',
  'Warwick-Edinbrugh',
  '\r'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  11045,
  10922,
  46291,
  11517,
  118,
  10912,
  60494,
  10104,
  19685,
  118,
  16627,
  27993,
  48234,
  118,
  13114,
  10245,
  79014,
  10237,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification

In [ ]:
import torch

In [ ]:
import transformers
import accelerate
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)

Transformers version: 4.40.2
Accelerate version: 0.30.1


In [ ]:
from transformers import TrainingArguments

In [ ]:

args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=5e-5,
per_device_train_batch_size=16,
# per_device_eval_batch_size=16,
num_train_epochs=3,
weight_decay=0.01,
)

In [ ]:
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     learning_rate=5e-5,
#     weight_decay=0.01
# )

In [ ]:
metric = datasets.load_metric("seqeval")

<ipython-input-27-b144b02b338d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
label_list = dataset.features["ner_tags"].feature.names

label_list


['O', 'B-QUESTION', 'I-QUESTION']

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }

In [ ]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_datasets,
   eval_dataset=tokenized_datasets,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     data_collator=DataCollatorForTokenClassification(tokenizer)
# )


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.126845,0.718186,0.839373,0.774065,0.955071
2,No log,0.042121,0.858412,0.888588,0.873239,0.982319
3,0.180800,0.022842,0.925469,0.933478,0.929456,0.991337


TrainOutput(global_step=546, training_loss=0.1686525401614961, metrics={'train_runtime': 250.6978, 'train_samples_per_second': 34.727, 'train_steps_per_second': 2.178, 'total_flos': 499780496772180.0, 'train_loss': 0.1686525401614961, 'epoch': 3.0})

Inference Code

In [ ]:
## Save model
model.save_pretrained("ner_model")

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [ ]:
id2label

{'0': 'O', '1': 'B-QUESTION', '2': 'I-QUESTION'}

In [ ]:
label2id

{'O': '0', 'B-QUESTION': '1', 'I-QUESTION': '2'}

In [ ]:
# from transformers import BertTokenizer, BertForTokenClassification
# import torch

In [ ]:
# # Saving the model
# model.save_pretrained('/content/my_model')

# # Saving the tokenizer
# tokenizer.save_pretrained('/content/my_model')

('/content/my_model/tokenizer_config.json',
 '/content/my_model/special_tokens_map.json',
 '/content/my_model/vocab.txt',
 '/content/my_model/added_tokens.json',
 '/content/my_model/tokenizer.json')

In [ ]:
import json

In [ ]:
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("./ner_model")

In [ ]:
model_fine_tuned

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [ ]:
from transformers import pipeline

In [ ]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "Q2.17 What is the home ownership status for you and/or your partner? We own our home outright  (1) We are paying off the mortgage on our home  (2) We are renting privately  (3) We are living with family or friends  (8) We are living in social or public housing  (9) Other (please specify)  (10) ________________________________________________\r"

ner_results = nlp(example)

print(ner_results)

[{'entity': 'B-QUESTION', 'score': 0.99708253, 'index': 5, 'word': 'What', 'start': 6, 'end': 10}, {'entity': 'I-QUESTION', 'score': 0.9970528, 'index': 6, 'word': 'is', 'start': 11, 'end': 13}, {'entity': 'I-QUESTION', 'score': 0.99808097, 'index': 7, 'word': 'the', 'start': 14, 'end': 17}, {'entity': 'I-QUESTION', 'score': 0.9978116, 'index': 8, 'word': 'home', 'start': 18, 'end': 22}, {'entity': 'I-QUESTION', 'score': 0.99820924, 'index': 9, 'word': 'ownership', 'start': 23, 'end': 32}, {'entity': 'I-QUESTION', 'score': 0.998654, 'index': 10, 'word': 'status', 'start': 33, 'end': 39}, {'entity': 'I-QUESTION', 'score': 0.99819046, 'index': 11, 'word': 'for', 'start': 40, 'end': 43}, {'entity': 'I-QUESTION', 'score': 0.9987251, 'index': 12, 'word': 'you', 'start': 44, 'end': 47}, {'entity': 'I-QUESTION', 'score': 0.9988502, 'index': 13, 'word': 'and', 'start': 48, 'end': 51}, {'entity': 'I-QUESTION', 'score': 0.99877566, 'index': 14, 'word': '/', 'start': 51, 'end': 52}, {'entity': 'I

In [ ]:
from transformers import pipeline

# Assuming model_fine_tuned and tokenizer are already defined
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "Q14. Os seus AMIGOS ou alguém próximo a você\r"

ner_results = nlp(example)

# Initialize variables to store the current question and all questions
current_question = []
all_questions = []

# Iterate through each entity in the NER results
for entity in ner_results:
    word = entity['word']
    entity_type = entity['entity']

    # Handle token continuation marked by '##'
    if word.startswith('##'):
        word = word[2:]  # Remove '##'
        if current_question:
            current_question[-1] += word  # Append directly to the last word
        continue

    # Handle new question starting
    if entity_type == 'B-QUESTION':
        # If there's an ongoing question, store it first
        if current_question:
            all_questions.append("".join(current_question).strip())
            current_question = []
        # Add the new word to start a fresh question
        current_question.append(word)
    elif entity_type == 'I-QUESTION':
        # Correct spacing issues for punctuation
        if word in {',', '.', ';', ':'}:
            if current_question and not current_question[-1].endswith(' '):
                current_question[-1] += word  # Append punctuation directly to the last word
        else:
            # Ensure proper spacing for words
            if current_question and not current_question[-1].endswith(' '):
                word = ' ' + word
            current_question.append(word)

# Append the last question if not empty
if current_question:
    all_questions.append("".join(current_question).strip())

# Print all questions
for question in all_questions:
    print(question)


In [ ]:
# saving files from colab

In [ ]:
import os
from zipfile import ZipFile

# Specify the folder you want to download
folder_path = '/content/ner_model'  # Change 'your_folder_name' to the name of your folder
zip_path = '/content/ner_model.zip'  # Output zip file name

# Creating a ZipFile object
with ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Create a complete filepath of file in directory
            file_path = os.path.join(root, file)
            # Add file to zip
            zipf.write(file_path, os.path.relpath(file_path, folder_path))
